In [1]:
import numpy as np
import pandas as pd
from pandas_datareader import data as wb
import matplotlib.pyplot as plt

C:\Users\BigData\Anaconda3\lib\site-packages\pandas_datareader\compat\__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [2]:
tickers = ['PG', 'BEI.DE']

sec_data = pd.DataFrame()

for t in tickers:
    sec_data[t] = wb.DataReader(t, data_source='yahoo', start='2007-1-1')['Adj Close']

In [3]:
sec_data.tail()

,PG,BEI.DE
Date,,
2020-06-17,117.930000,100.250000
2020-06-18,119.279999,99.760002
2020-06-19,118.919998,101.250000
2020-06-22,117.750000,99.800003
2020-06-23,118.580002,101.199997


In [4]:
sec_returns = np.log(sec_data / sec_data.shift(1))

In [5]:
sec_returns

,PG,BEI.DE
Date,,
2007-01-03,NaN,NaN
2007-01-04,-0.007621,0.006544
2007-01-05,-0.008624,-0.020772
2007-01-08,0.002202,0.000202
2007-01-09,-0.002517,-0.022858
...,...,...
2020-06-17,-0.001694,0.003497
2020-06-18,0.011382,-0.004900
2020-06-19,-0.003023,0.014825


## PG

In [6]:
sec_returns['PG'].mean()

0.0002981314770183057

In [7]:
sec_returns['PG'].mean() * 250

0.07453286925457643

In [8]:
sec_returns['PG'].std()

0.011963899145740139

In [9]:
sec_returns['PG'].std() * 250 ** 0.5

0.189165854985408

## Beiersdorf

In [10]:
sec_returns['BEI.DE'].mean()

0.00025998560374800776

In [11]:
sec_returns['BEI.DE'].mean() * 250

0.06499640093700194

In [12]:
sec_returns['BEI.DE'].std()

0.013801747280467926

In [13]:
sec_returns['BEI.DE'].std() * 250 ** 0.5

0.21822478548156704

***

In [14]:
print (sec_returns['PG'].mean() * 250)
print (sec_returns['BEI.DE'].mean() * 250)

0.07453286925457643
0.06499640093700194


In [15]:
sec_returns[['PG', 'BEI.DE']].mean() * 250

PG        0.074533
BEI.DE    0.064996
dtype: float64

In [16]:
sec_returns[['PG', 'BEI.DE']].std() * 250 ** 0.5

PG        0.189166
BEI.DE    0.218225
dtype: float64

## Covariance and Correlation


\begin{eqnarray*}
Covariance Matrix: \  \   
\Sigma = \begin{bmatrix}
        \sigma_{1}^2 \ \sigma_{12} \ \dots \ \sigma_{1I} \\
        \sigma_{21} \ \sigma_{2}^2 \ \dots \ \sigma_{2I} \\
        \vdots \ \vdots \ \ddots \ \vdots \\
        \sigma_{I1} \ \sigma_{I2} \ \dots \ \sigma_{I}^2
    \end{bmatrix}
\end{eqnarray*}

In [17]:
PG_var = sec_returns['PG'].var() 
PG_var

0.00014313488276944164

In [18]:
BEI_var = sec_returns['BEI.DE'].var() 
BEI_var

0.00019048822799390378

In [19]:
PG_var_a = sec_returns['PG'].var() * 250
PG_var_a

0.03578372069236041

In [20]:
BEI_var_a = sec_returns['BEI.DE'].var() * 250
BEI_var_a

0.047622056998475944

***

In [21]:
cov_matrix = sec_returns.cov()
cov_matrix

,PG,BEI.DE
PG,0.000143,0.000046
BEI.DE,0.000046,0.000190


In [22]:
cov_matrix_a = sec_returns.cov() * 250
cov_matrix_a

,PG,BEI.DE
PG,0.035784,0.011382
BEI.DE,0.011382,0.047622


***

In [23]:
corr_matrix = sec_returns.corr()
corr_matrix

,PG,BEI.DE
PG,1.00000,0.27532
BEI.DE,0.27532,1.00000


## Calculating Portfolio Risk

Equal weigthing scheme:

In [24]:
weights = np.array([0.5, 0.5])

Portfolio Variance:

In [25]:
pfolio_var = np.dot(weights.T, np.dot(sec_returns.cov() * 250, weights))
pfolio_var

0.026542673140761537

Portfolio Volatility:

In [26]:
pfolio_vol = (np.dot(weights.T, np.dot(sec_returns.cov() * 250, weights))) ** 0.5
pfolio_vol

0.1629192227478438

In [27]:
print (str(round(pfolio_vol, 5) * 100) + ' %')

16.292 %


## Calculating Diversifiable and Non-Diversifiable Risk of a Portfolio

In [28]:
weights = np.array([0.5, 0.5])

***

Diversifiable Risk:

In [31]:
PG_var_a = sec_returns[['PG']].var() * 250
PG_var_a

PG    0.035784
dtype: float64

In [32]:
BEI_var_a = sec_returns[['BEI.DE']].var() * 250
BEI_var_a

BEI.DE    0.047622
dtype: float64

$$
\text{Diversifiable risk} = \text{Portfolio variance} - \text{Weighted annual variances}
$$

In [33]:
dr = pfolio_var - (weights[0] ** 2 * PG_var_a) - (weights[1] ** 2 * BEI_var_a)
dr

BEI.DE   NaN
PG       NaN
dtype: float64

In [34]:
float(PG_var_a)

0.03578372069236041

In [35]:
PG_var_a = sec_returns['PG'].var() * 250
PG_var_a

0.03578372069236041

In [36]:
BEI_var_a = sec_returns['BEI.DE'].var() * 250
BEI_var_a

0.047622056998475944

In [37]:
dr = pfolio_var - (weights[0] ** 2 * PG_var_a) - (weights[1] ** 2 * BEI_var_a)
dr

0.005691228718052449

In [38]:
print (str(round(dr*100, 3)) + ' %')

0.569 %


Non-Diversifiable Risk:

In [39]:
n_dr_1 = pfolio_var - dr
n_dr_1

0.020851444422709087

In [40]:
n_dr_2 = (weights[0] ** 2 * PG_var_a) + (weights[1] ** 2 * BEI_var_a)
n_dr_2

0.020851444422709087

In [41]:
n_dr_1 == n_dr_2

True